In [4]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [51]:
import yfinance as yf
import pandas as pd

In [52]:
tickers= pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol

In [53]:
tickers=tickers.to_list()

In [54]:
price_df=yf.download(tickers,start='2010-01-01')

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No timezone found, symbol may be delisted


In [55]:
price_df[price_df.columns[price_df.columns.get_level_values(1)=='AAPL']]

,Adj Close,Close,High,Low,Open,Volume
,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2010-01-04,6.505280,7.643214,7.660714,7.585000,7.622500,493729600
2010-01-05,6.516528,7.656429,7.699643,7.616071,7.664286,601904800
2010-01-06,6.412873,7.534643,7.686786,7.526786,7.656429,552160000
2010-01-07,6.401018,7.520714,7.571429,7.466071,7.562500,477131200
2010-01-08,6.443574,7.570714,7.571429,7.466429,7.510714,447610800
...,...,...,...,...,...,...
2023-05-05,173.570007,173.570007,174.300003,170.759995,170.979996,113316400
2023-05-08,173.500000,173.500000,173.850006,172.110001,172.479996,55962800


In [56]:
def slice_df(symbol):
    sliced= price_df.copy()
    sliced=sliced[sliced.columns[sliced.columns.get_level_values(1)==symbol]]
    sliced.columns = sliced.columns.droplevel(1)
    sliced.loc[:,'price']=sliced.Open.shift(-1)
    
    return sliced

In [57]:
test=slice_df('AAPL')

In [58]:
def ma_calc(df,n,m):
    df['sma_1'] = df.Close.rolling(n).mean()
    df['sma_2'] = df.Close.rolling(m).mean()

In [59]:
ma_calc(test,50,100)

In [60]:
def backtest(df,n,m):
    ma_calc(df,n,m)
    in_position =False
    profits = []
    
    for index,row in df.iterrows():
        if not in_position:
            if row.sma_1> row.sma_2:
                buyprice= row.price
                in_position = True
        if in_position:
            if row.sma_1< row.sma_2:
                profit=(row.price - buyprice)/buyprice
                profits.append(profit)
                in_position= False
    gain = (pd.Series(profits)+1).prod()
    return gain

In [61]:
backtest(test,50,10)

2.2069046522319926

In [62]:
results=[]

for sym in tickers:
    subdf= slice_df(sym)
    print("Result for "+ sym)
    print(backtest(subdf,50,10))
    results.append(backtest(subdf,50,10))

Result for MMM
1.6578688041390162
Result for AOS
3.3487306896354165
Result for ABT
2.5302180044900062
Result for ABBV
2.200228365306093
Result for ACN
4.367416856026402
Result for ATVI
2.2451994166089713
Result for ADM
1.2860598731606068
Result for ADBE
3.272337441212838
Result for ADP
2.7192671854880635
Result for AAP
0.9669009862591116
Result for AES
1.4855517049974005
Result for AFL
2.2534144601431665
Result for A
2.375910749084092
Result for APD
2.7802015915910907
Result for AKAM
2.675780732830036
Result for ALK
5.039299810157898
Result for ALB
2.3347075208698684
Result for ARE
2.4313437382021132
Result for ALGN
0.5910316909772849
Result for ALLE
2.2108442873867324
Result for LNT
2.6188093580870975
Result for ALL
1.3656559054972357
Result for GOOGL
2.641699876814933
Result for GOOG
2.738336288008509
Result for MO
1.0563076331086838
Result for AMZN
2.3411152271533733
Result for AMCR
1.8085613604629531
Result for AMD
1.1557424808513677
Result for AEE
3.3313843024727188
Result for AAL

In [47]:
profits = pd.DataFrame({'profit':results},index=tickers)

In [48]:
(slice_df('HUM').Close.pct_change()+1).prod()

11.732925096148165

In [50]:
(slice_df('HUM').Close).plot()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [38]:
profits.nlargest(50,'profit')

,profit
HUM,17.253224
BKNG,11.208926
ULTA,9.417876
AAL,9.284522
MOH,9.262711
ADI,8.861906
UNH,8.471105
KLAC,8.167045
TDY,7.906273
UAL,7.776383
